In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as rms
from sklearn.metrics import roc_auc_score
from math import exp

In [2]:
df = pd.read_csv('data-logistic.csv', header=None)
y = df[0]
X = df.loc[:, 1:]

Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).


Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [3]:
K = 0.1
ERROR = 1e-5

def sigma_y(X, y, i, w1, w2):
    return 1. / (1. + exp(-y[i] * (w1*X[1][i] + w2*X[2][i])))

def delta_for_w(w_index, w1, w2, C, X, y):
    addition = sum((
        y[i] * X[w_index][i] * (1. - sigma_y(X, y, i, w1, w2)) for i in np.arange(0, len(y))
    ))
    addition *= K / len(y)
    addition -= K * C * (w1 if w_index == 1 else w2)
    
    return addition

In [4]:
def gradient_regressor(X, y, C, iterations_remaining=10000):
    changed_w1, changed_w2 = 0., 0.
    while iterations_remaining:
        iterations_remaining -= 1
        w1, w2 = changed_w1, changed_w2
        changed_w1 = w1 + delta_for_w(1, w1, w2, C, X, y)
        changed_w2 = w2 + delta_for_w(2, w1, w2, C, X, y)
        if np.sqrt(rms([w1, w2], [changed_w1, changed_w2])) <= ERROR:
            break
    return changed_w1, changed_w2

def sigma(xi, w1, w2):
    return 1. / (1 + np.exp(-w1 * xi[1] - w2 * xi[2]))

In [8]:
w1, w2 = gradient_regressor(X, y, 0.)
l2w1, l2w2 = gradient_regressor(X, y, 10.)

scores = X.apply(lambda xi: sigma(xi, w1, w2), axis=1)
l2scores = X.apply(lambda xi: sigma(xi, l2w1, l2w2), axis=1)

Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. 

Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2))

In [7]:
auc_score = round(roc_auc_score(y, scores), 3)
l2_auc_score = round(roc_auc_score(y, l2scores), 3)

print(auc_score)
print(l2_auc_score)

f = open('submission.txt', 'w')
f.write(str(auc_score) + ' ' + str(l2_auc_score))
f.close()

0.927
0.936
